In [1]:
import os
import shutil
import random
from sklearn.model_selection import train_test_split

#  CONFIGURATION
image_dir = "/kaggle/input/original/train/images"
label_dir = "/kaggle/input/original/train/labels"
bg_image_dir = "/kaggle/input/background/background_only"
bg_label_dir = "/kaggle/input/background/background_only_labels"
output_base = "/kaggle/working/combined_split"
num_bg_to_add = 180  # 👈 change this as needed

#  Get only labeled image base names (non-empty label .txts)
image_basenames = []
for fname in os.listdir(label_dir):
    if fname.endswith(".txt"):
        with open(os.path.join(label_dir, fname)) as f:
            if f.read().strip():
                image_basenames.append(fname.replace(".txt", ""))

#  Split labeled data
train_val, test = train_test_split(image_basenames, test_size=0.1, random_state=42)
train, val = train_test_split(train_val, test_size=0.2222, random_state=42)  # ~20% val

#  Prepare output folders
def copy_split(split_names, split_type):
    img_out = os.path.join(output_base, split_type, "images")
    lbl_out = os.path.join(output_base, split_type, "labels")
    os.makedirs(img_out, exist_ok=True)
    os.makedirs(lbl_out, exist_ok=True)

    for name in split_names:
        shutil.copy(os.path.join(image_dir, f"{name}.jpg"), os.path.join(img_out, f"{name}.jpg"))
        shutil.copy(os.path.join(label_dir, f"{name}.txt"), os.path.join(lbl_out, f"{name}.txt"))

copy_split(train, "train")
copy_split(val, "valid")
copy_split(test, "test")

#  Select & copy a limited number of background-only images to training set
bg_images = [f for f in os.listdir(bg_image_dir) if f.endswith(".jpg")]
random.shuffle(bg_images)
bg_images_to_add = bg_images[:num_bg_to_add]

train_img_dir = os.path.join(output_base, "train/images")
train_lbl_dir = os.path.join(output_base, "train/labels")

for img_name in bg_images_to_add:
    base = os.path.splitext(img_name)[0]
    img_path = os.path.join(bg_image_dir, img_name)
    lbl_path = os.path.join(bg_label_dir, f"{base}.txt")
    shutil.copy(img_path, os.path.join(train_img_dir, img_name))
    shutil.copy(lbl_path, os.path.join(train_lbl_dir, f"{base}.txt"))

print(f"✅ Final split: {len(train)} train (+{len(bg_images_to_add)} bg), {len(val)} val, {len(test)} test")

✅ Final split: 346 train (+180 bg), 99 val, 50 test


In [2]:
yaml_content = """
train: /kaggle/working/combined_split/train/images
val: /kaggle/working/combined_split/valid/images
test: /kaggle/working/combined_split/test/images

nc: 4
names: ['Absence_Head', 'Absence_Tail', 'Head_Rivet', 'Tail_Rivet']
"""

yaml_path = "/kaggle/working/rivet_detection.yaml"
with open(yaml_path, "w") as f:
    f.write(yaml_content.strip())

print(f"✅ YAML file written to: {yaml_path}")

✅ YAML file written to: /kaggle/working/rivet_detection.yaml


In [ ]:
!pip install ultralytics --upgrade --quiet

import os
from ultralytics import YOLO


DATA_YAML_PATH = "/kaggle/working/rivet_detection.yaml"  
model = YOLO("yolov5s.pt").to("cuda")  

# Train the model
model.train(
    data=DATA_YAML_PATH,
    device='cuda',
    epochs=100,
    batch=8,
    imgsz=896,         
    patience=20,
    optimizer='SGD',
    lr0=0.002,
    lrf=0.1,
    weight_decay=0.0005,
    cos_lr=True,
    save_period=10,
    workers=4,
    amp=True,

    mosaic=1.0,
    auto_augment='randaugment',
    augment=True,

    hsv_h=0.01,
    hsv_s=0.8,
    hsv_v=0.4,

    flipud=0.5,
    fliplr=0.5,

    translate=0.1,
    scale=0.3,
    shear=0.05,
    perspective=0.001,
    degrees=5.0,

    mixup=0.1,
    erasing=0.0,
    close_mosaic=2,
    dropout=0.1,
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.3 MB/s eta 0:00:00:00:0100:01
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.c

100%|██████████| 17.7M/17.7M [00:00<00:00, 158MB/s]


Ultralytics 8.3.157 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=2, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/kaggle/working/rivet_detection.yaml, degrees=5.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.1, dynamic=False, embed=None, epochs=100, erasing=0.0, exist_ok=False, fliplr=0.5, flipud=0.5, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.01, hsv_s=0.8, hsv_v=0.4, imgsz=896, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.002, lrf=0.1, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=yolov5s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=SGD, overlap_mask=True, patience=20, perspective=0.001, plots=True, pose=12.0, pretraine

100%|██████████| 755k/755k [00:00<00:00, 17.3MB/s]


Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1      3520  ultralytics.nn.modules.conv.Conv             [3, 32, 6, 2, 2]              
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     18816  ultralytics.nn.modules.block.C3              [64, 64, 1]                   
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    115712  ultralytics.nn.modules.block.C3              [128, 128, 2]                 
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  3    625152  ultralytics.nn.modules.block.C3              [256, 256, 3]                 
  7                  -1  1   1180672  ultralytics

...

train: Scanning /kaggle/working/combined_split/train/labels... 526 images, 180 backgrounds, 0 corrupt: 100%|██████████| 526/526 [00:00<00:00, 1483.50it/s]

train: New cache created: /kaggle/working/combined_split/train/labels.cache


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1297.9±973.1 MB/s, size: 340.7 KB)


val: Scanning /kaggle/working/combined_split/valid/labels... 99 images, 0 backgrounds, 0 corrupt: 100%|██████████| 99/99 [00:00<00:00, 1634.52it/s]

val: New cache created: /kaggle/working/combined_split/valid/labels.cache


In [4]:
!pip install ultralytics --upgrade --quiet
import os
import shutil
from ultralytics import YOLO
import matplotlib.pyplot as plt

# Step 1: Define paths
weights_path = "/kaggle/input/weight-yolov5/best.pt"  # Change this to your actual path
predict_dir = "/kaggle/working/predict"
test_images_dir = "/kaggle/working/combined_split/test/images"
yaml_path = "/kaggle/working/rivet_detection.yaml"

# Step 2: Create predict folder
os.makedirs(predict_dir, exist_ok=True)

# Step 3: Copy weights to working directory if needed
working_weights_path = "/kaggle/input/weight-yolov5/best.pt"
if not os.path.exists(working_weights_path):
    shutil.copy(weights_path, working_weights_path)

# Step 4: Load the model
model = YOLO(working_weights_path)

# Step 5: Run prediction on test images
results = model.predict(
    source=test_images_dir,
    save=True,
    save_txt=True,
    conf=0.50,
    imgsz=896,
    project=predict_dir,
    name="predict_results",
    exist_ok=True
)

# Step 6: Run validation on test set to get confusion matrix
metrics = model.val(
    data=yaml_path,
    split='test',
    save_json=True,
    plots=True,
    project=predict_dir,
    name="val_results",
    exist_ok=True
)

print("✅ Inference complete.")


image 1/50 /kaggle/working/combined_split/test/images/IMG-20250525-WA0005_jpg.rf.2dd9fc700d1f491869c790bc6592912e_aug_1.jpg: 896x896 1 Absence_Tail, 6 Head_Rivets, 5 Tail_Rivets, 24.0ms
image 2/50 /kaggle/working/combined_split/test/images/IMG-20250525-WA0005_jpg.rf.2dd9fc700d1f491869c790bc6592912e_aug_3.jpg: 896x896 1 Absence_Tail, 6 Head_Rivets, 5 Tail_Rivets, 24.0ms
image 3/50 /kaggle/working/combined_split/test/images/IMG-20250525-WA0007_jpg.rf.36efe4178203e6b12bc53223001f5c4e.jpg: 896x896 1 Absence_Tail, 6 Head_Rivets, 5 Tail_Rivets, 24.0ms
image 4/50 /kaggle/working/combined_split/test/images/IMG-20250525-WA0007_jpg.rf.36efe4178203e6b12bc53223001f5c4e_aug_2.jpg: 896x896 1 Absence_Tail, 6 Head_Rivets, 5 Tail_Rivets, 23.9ms
image 5/50 /kaggle/working/combined_split/test/images/IMG-20250525-WA0009_jpg.rf.bc224a81eb1b115382dd6b58dc81aad5.jpg: 896x896 6 Head_Rivets, 6 Tail_Rivets, 23.9ms
image 6/50 /kaggle/working/combined_split/test/images/IMG-20250525-WA0009_jpg.rf.bc224a81eb1b1153

100%|██████████| 755k/755k [00:00<00:00, 17.2MB/s]

val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2778.7±632.4 MB/s, size: 651.1 KB)



val: Scanning /kaggle/working/combined_split/test/labels... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<00:00, 668.04it/s]

val: New cache created: /kaggle/working/combined_split/test/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:05<00:00,  1.30s/it]
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all         50        600      0.993       0.96      0.991      0.649
          Absence_Head          7          7          1      0.849      0.978      0.592
          Absence_Tail         20         20      0.976          1      0.995      0.677
            Head_Rivet         50        292      0.998      0.997      0.995      0.691
            Tail_Rivet         50        281      0.997      0.993      0.995      0.637
Speed: 11.6ms preprocess, 23.4ms inference, 0.0ms loss, 18.0ms postprocess per image
Saving /kaggle/working/predict/val_results/predictions.json...
Results saved to /kaggle/working/predict/val_results
✅ Inference complete.


In [5]:
import shutil

# Zip the predict_tta folder
shutil.make_archive("/kaggle/working/predict/predict_results", 'zip', "/kaggle/working/predict")
print("✅ Folder zipped as /kaggle/working/predict.zip")

✅ Folder zipped as /kaggle/working/predict.zip
